In [1]:
import gym
import torch
import torch.nn as nn
from torch.nn.parameter import Parameter
import torch.optim as optim
import numpy as np
import copy
import time
from gym.envs.toy_text.frozen_lake import FrozenLakeEnv
env =FrozenLakeEnv(is_slippery=False,map_name="4x4")
#env =FrozenLakeEnv(is_slippery=False,map_name="8x8")
env.render()


SFFF
FHFH
FFFH
HFFG


In [2]:
class Net(nn.Module):
    def __init__(self,para,bias=True):
        super(Net,self).__init__()
        self.input_dim=para['input_dim']
        self.output_dim=para['output_dim']
        self.hidden_dim=para['hidden_dim']
        
        self.w1=Parameter(torch.FloatTensor(self.input_dim,self.hidden_dim))
        self.b1=Parameter(torch.FloatTensor(self.hidden_dim))
        nn.init.kaiming_normal_(self.w1)
#        nn.init.kaiming_uniform_(self.w1)
        nn.init.uniform_(self.b1,-0.001,0.001)

        self.w2=Parameter(torch.FloatTensor(self.hidden_dim,self.hidden_dim))
        self.b2=Parameter(torch.FloatTensor(self.hidden_dim))
        nn.init.kaiming_normal_(self.w2)
#        nn.init.kaiming_uniform_(self.w2)
        nn.init.uniform_(self.b2,-0.001,0.001)
        
        self.w3=Parameter(torch.FloatTensor(self.hidden_dim,self.hidden_dim))
        self.b3=Parameter(torch.FloatTensor(self.hidden_dim))
        nn.init.kaiming_normal_(self.w3)
#        nn.init.kaiming_uniform_(self.w3)
        nn.init.uniform_(self.b3,-0.001,0.001)
        
        self.w4=Parameter(torch.FloatTensor(self.hidden_dim,self.output_dim))
        self.b4=Parameter(torch.FloatTensor(self.output_dim))
        nn.init.kaiming_normal_(self.w4)
#        nn.init.kaiming_uniform_(self.w4)
        nn.init.uniform_(self.b4,-0.001,0.001)
        
    def forward(self,X):
        
        H=torch.relu( torch.einsum('li,ij->lj',X,self.w1)+self.b1)
        H=torch.relu( torch.einsum('li,ij->lj',H,self.w2)+self.b2)
        H=torch.relu( torch.einsum('li,ij->lj',H,self.w3)+self.b3)
        Y=torch.einsum('li,ij->lj',H,self.w4)+self.b4
        Y=torch.softmax(Y,dim=1)

        return Y

    

In [3]:
input_dim=env.observation_space.n
output_dim=env.action_space.n
hidden_dim=100
lr=0.0001
lr2=1.0

para={'input_dim': input_dim, 'output_dim': output_dim, 'hidden_dim': hidden_dim}
model=Net(para)
#criterion=nn.MSELoss()
#criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr)

In [7]:
max_episodes=100
max_step=100

gamma=0.99
epsilon0 = 1.0


# initialize

epsilon=epsilon0
min_step=max_step
min_step_episode=max_episodes
min_step_path=[]

#scores=[]
episodes=[]
for episode in range(max_episodes):
    epsilon=epsilon*0.99
    state = env.reset()
    total_reward = 0
    state_list = [state]
    win=0
    samples=[]
    for step in range(0,max_step):
        state_m=np.zeros([1,env.observation_space.n],dtype=np.float32)
        state_m[0][state]=1.0
        state_m_torch=torch.from_numpy(state_m)
        out_softmax=model(state_m_torch)
#        print(state)
#        print(out_softmax)
        policy=out_softmax.cpu().detach().numpy()[0]
        action=np.random.choice(env.action_space.n,p=policy)

        state_next,reward,game_over,_ = env.step(action)
#        print(state_next)
        if reward==0 : 
            reward=-0.1
        if reward!=1.0 and game_over==1:
            reward=-1.0

        samples+=[[state,action,reward]]
#        scores+=reward
        
        state=copy.deepcopy(state_next)
        
#        state_m_next=np.zeros([1,env.observation_space.n],dtype=np.float32)
#        state_m_next[0][state_next]=1.0
        
        if game_over or step==max_step-1: 
            print(step)

            rr=0
            Nsamples=len(samples)
            Gt=np.zeros([Nsamples,1],dtype=np.float32)
            states_m=np.zeros([Nsamples,env.observation_space.n],dtype=np.float32)
#            action_m=np.zeros([Nsamples],dtype=np.long)
            actions_m=np.zeros([Nsamples,env.action_space.n],dtype=np.float32)
            for i in range(Nsamples):
#                print(samples[i])
                state=samples[i][0]
                action=samples[i][1]
                reward=samples[i][2]
                rr= rr*gamma+reward
                Gt[i]=rr
                states_m[i][state]=1.0
                actions_m[i][action]=1.0
#                actions_m[i]=action

            states_m_torch=torch.from_numpy(states_m)
            actions_m_torch=torch.from_numpy(actions_m)
            Gt_torch=torch.from_numpy(Gt)

        
            optimizer.zero_grad()
            out_softmax=model(states_m_torch)
            loss=-torch.sum(actions_m_torch*torch.log(out_softmax)*Gt_torch)
            loss.backward(retain_graph=True)
            optimizer.step()
            cost=loss.data
            print(episode,step,cost)

        total_reward=total_reward+reward
        state_list.append(state)
        if game_over and reward==1:
            win=1
            if step< min_step:
                min_step=step
                min_step_path=state_list
                min_step_episode=episode
        if game_over==1:
            break


    line_out="Episode: %d, step: %d, total_reward: %f, epsilon: %f" %(episode, step, total_reward,epsilon)
    print(line_out)
    if reward==1 :
        print(state_list)
print( "min step: ", min_step, "min step episode", min_step_episode)
print("shortest path: ", min_step_path)


39
0 39 tensor(-53.6139)
Episode: 0, step: 39, total_reward: -4.900000, epsilon: 0.990000
28
1 28 tensor(-46.4481)
Episode: 1, step: 28, total_reward: -3.800000, epsilon: 0.980100
45
2 45 tensor(-84.1848)
Episode: 2, step: 45, total_reward: -5.500000, epsilon: 0.970299
17
3 17 tensor(-21.2413)
Episode: 3, step: 17, total_reward: -2.700000, epsilon: 0.960596
99
4 99 tensor(-292.1635)
Episode: 4, step: 99, total_reward: -10.000000, epsilon: 0.950990
99
5 99 tensor(-287.7385)
Episode: 5, step: 99, total_reward: -10.000000, epsilon: 0.941480
46
6 46 tensor(-56.9339)
Episode: 6, step: 46, total_reward: -5.600000, epsilon: 0.932065
94
7 94 tensor(-236.7540)
Episode: 7, step: 94, total_reward: -10.400000, epsilon: 0.922745
39
8 39 tensor(-66.3811)
Episode: 8, step: 39, total_reward: -4.900000, epsilon: 0.913517
99
9 99 tensor(-275.0953)
Episode: 9, step: 99, total_reward: -10.000000, epsilon: 0.904382
99
10 99 tensor(-142.8928)
Episode: 10, step: 99, total_reward: -10.000000, epsilon: 0.89533

25
96 25 tensor(-41.9502)
Episode: 96, step: 25, total_reward: -3.500000, epsilon: 0.377237
30
97 30 tensor(-39.4309)
Episode: 97, step: 30, total_reward: -4.000000, epsilon: 0.373464
99
98 99 tensor(-317.4319)
Episode: 98, step: 99, total_reward: -10.000000, epsilon: 0.369730
35
99 35 tensor(-57.0469)
Episode: 99, step: 35, total_reward: -4.500000, epsilon: 0.366032
min step:  100 min step episode 100
shortest path:  []


In [5]:
#state=np.arange(0,15)
#sess.run(init)
state_m=np.zeros([env.observation_space.n,env.observation_space.n],dtype=np.float32)
for i in range(0,env.observation_space.n):
    state_m[i][i]=1.0
print(state_m)
state_m_torch=torch.from_numpy(state_m)
Q_pred=torch.softmax(model(state_m_torch),dim=1)
Q_pred=Q_pred.cpu().detach().numpy()
#print(Q_pred)
for i in range(0,env.observation_space.n):
    line=""
    for j in range(0,env.action_space.n):
        line+="%6.3f " %(Q_pred[i][j])
#    line+="\n"
    print(line)
    

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
 0.348  0.215  0.204  0.233 
 0.380  0.191  0.196  0.233 
 0.352  0.221  0.201  0.226 
 0.222  0.198  0.200  0.380 
 0.260  0.222  0.200  0.318 
 0.223  0.211  0.194  0.373 
 0.381  0